In [1]:
import sys
import os

In [2]:
import pandas as pd
import numpy as np

/Users/jasreenmehta/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Either hard code your path to the project, or set it as an environment variable
from dotenv import load_dotenv
load_dotenv()  # this reads the .env file and updates os.environ

build_project_path = os.environ['BUILD_PROJECT_PATH']

In [4]:
synthetic_data_path = os.path.join(build_project_path, 'synthetic_data')
data_path = os.path.join(synthetic_data_path, 'data')

In [5]:
if synthetic_data_path not in sys.path:
    sys.path.append(synthetic_data_path)

In [17]:
from llm_requests import generate_prompt

seed_titles_df = pd.read_csv(os.path.join(data_path, 'seed_titles.csv'))

seed_titles = seed_titles_df['seed_title'].unique()

example_seed_titles = np.random.choice(seed_titles, 5)

example_prompt = generate_prompt(example_seed_titles)

<h3>Experiment with different models and see how they compare!</h3>

In [28]:
#model_name = 'Meta-Llama-3.3-70B-Instruct'
model_name = 'Meta-Llama-3.1-405B-Instruct'

In [29]:
#api_key=os.environ["SAMBANOVA_API_KEY"]

api_key = os.environ.get("SAMBANOVA_API_KEY")
print("API key found?" , bool(api_key))  # should print True

from llm_requests import get_client

client = get_client()

API key found? True


In [31]:
from llm_requests import async_make_api_call

example_response = await async_make_api_call(client, model_name, example_prompt, perturbation_std=0.1)

In [32]:
from gpt_parsing import parse_gpt_response

parsed_output = parse_gpt_response(example_response.choices[0].message.content, 5, 5)

In [33]:
for seed_title, response in zip(example_seed_titles, parsed_output):
    print(f'Variations of: {seed_title}:')
    print('-------------------')
    for i, variation in enumerate(response):
        print(f'{i+1}: {variation}')

    print('\n\n')

Variations of: Grievance Manager:
-------------------
1: Director, Employee Relations and Grievance Manager - Human Resources
2: Grievance Manager - Union & Non-Union Employee Relations
3: Compliance and Grievance Manager - Senior Role in Healthcare Industry
4: Grievance and Appeals Manager - Medicare/Medicaid Experience Required
5: Senior Employee Relations Specialist & Grievance Manager - East Coast Operations



Variations of: Range Management Specialist:
-------------------
1: Range Management Specialist - US Forest Service Career Opportunity
2: Senior Range Management Specialist - Bureau of Land Management
3: Range and Wildlife Management Specialist - Conservation Organization
4: Natural Resource Specialist (Range Management) - Government Agency
5: Range Management Specialist Wanted - Work Outdoors in Beautiful Landscapes



Variations of: Car Audio Installer:
-------------------
1: Experienced Car Audio Installer Wanted - High-End Custom Shop
2: Mobile Electronics and Car Audio I

In [34]:
from llm_requests import async_main_stubborn

output_dict_path = os.path.join(data_path, 'jitter_responses.pkl')
response_dict = await async_main_stubborn(
    all_query_titles=seed_titles,
    client=client,
    model_name=model_name,
    output_path=output_dict_path,
    chunk_size=5,
    num_examples_per_title=5,
    giveup_after=1,
)

  0%|          | 0/1169 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'code': None, 'message': 'Rate limit exceeded', 'param': None, 'type': 'rate_limit_exceeded'}}

In [7]:
import pickle

output_dict_path = os.path.join(data_path, 'jitter_responses.pkl')

with open(output_dict_path, 'rb') as f:
    response_dict = pickle.load(f)

In [ ]:
jitter_df = {
    'jittered_title': [],
    'seed_title': [],
}

for seed_title, jittered_titles in response_dict.items():
    for jittered_title in jittered_titles:
        jitter_df['jittered_title'].append(jittered_title)
        jitter_df['seed_title'].append(seed_title)

jitter_df = pd.DataFrame(jitter_df)

jitter_df = jitter_df.merge(seed_titles_df, on='seed_title', how='left')
jitter_df.sample(5)

In [9]:
jitter_df.to_csv(os.path.join(data_path, 'jittered_titles.csv'), index=False)